In [1]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 34.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
pathos 0.3.1 requires dill>=0.3.7, but you have dill 0.3.6 which is incompatible.
pathos 0.3.1 requires multiprocess>=0.70.15, but you have multiprocess 0.70.14 which is incompatible.
pymc3 3.11.5 requires numpy<1.22.2,>=1.15.0, but you have numpy 1.23.5 which i

In [2]:
from datasets import load_from_disk, Dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# using GPU if you have one
device = "cuda:0" if torch.cuda.is_available() else "cpu"
# device = "cpu"

## 1. Load Dataset and Prepare Data for Weak Supervision

In [4]:
dataset = load_from_disk("../input/chat-weak/chat_weak")

In [5]:
dataset.save_to_disk("/kaggle/working/chat_weak")

Saving the dataset (0/1 shards):   0%|          | 0/15098 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/23421 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/18729 [00:00<?, ? examples/s]

In [6]:
dataset = load_from_disk("/kaggle/working/chat_weak")

## 2. Load Model

In [7]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

## 2. Instruction Fine-tuning

In [8]:
def tokenize_function(example):
    start_prompt = 'Evaluate the sentiment of the following sentence.\n\n'
    end_prompt = '\n\nSentiment: '
    prompt = [start_prompt + message + end_prompt for message in example["message"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["sentiment"], padding="max_length", truncation=True, return_tensors="pt").input_ids
    
    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['conversation_id', 'message', 'sentiment', '__index_level_0__', 'speaker_id', ])


Map:   0%|          | 0/15098 [00:00<?, ? examples/s]

Map:   0%|          | 0/23421 [00:00<?, ? examples/s]

Map:   0%|          | 0/18729 [00:00<?, ? examples/s]

In [9]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['val'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)


Shapes of the datasets:
Training: (15098, 2)
Validation: (23421, 2)
Test: (18729, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 15098
    })
    val: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 23421
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 18729
    })
})


### 2.1 Fine-tune Model

In [12]:
output_dir = f'/kaggle/working/model/instruction-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=500,
    evaluation_strategy = "steps",
    save_total_limit=1,
    load_best_model_at_end=True,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['val']
)

In [13]:
torch.cuda.empty_cache()

In [14]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
cat: /sys/module/amdgpu/initstate: No such file or directory


Step,Training Loss,Validation Loss
500,36.701500,38.341583
1000,31.868200,35.458488
1500,30.633700,34.864864
2000,30.272700,34.769234
2500,30.141200,34.741844
3000,30.223000,34.712502
3500,30.124200,34.705189
4000,30.156000,34.695347
4500,30.138500,34.692413
5000,30.090300,34.693790


TrainOutput(global_step=7549, training_loss=30.744005828586566, metrics={'train_runtime': 27858.8562, 'train_samples_per_second': 0.542, 'train_steps_per_second': 0.271, 'total_flos': 1.0338466507259904e+16, 'train_loss': 30.744005828586566, 'epoch': 1.0})

In [16]:
import zipfile
import os
from IPython.display import FileLink

def zip_dir(directory = os.curdir, file_name = 'directory.zip'):
    """
    zip all the files in a directory
    
    Parameters
    _____
    directory: str
        directory needs to be zipped, defualt is current working directory
        
    file_name: str
        the name of the zipped file (including .zip), default is 'directory.zip'
        
    Returns
    _____
    Creates a hyperlink, which can be used to download the zip file)
    """
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))

    return FileLink(file_name)

In [17]:
zip_dir()

/kaggle/working/directory.zip